## Set Up

We import our code and any frequently used libraries, and set up our data.

In [172]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [176]:
from kernels import polynomial_kernel
from perceptrons import OneVsAllKernelPerceptron, KernelPerceptron, SpeedyKernelPerceptron

<b> Let's initially work with a smaller dataset until we sort out inner and outer efficiency issues (i.e. using one-vs-all and making the individual perceptrons more efficient) <b>

In [177]:
df = pd.read_csv('zipcombo.dat', sep=' ', header=None).drop(columns=[257])
df.rename(columns={0: 'label'}, inplace=True)
X = df[list(range(1, 257))].values
y = df['label'].values.astype(np.int)

In [178]:
def subsample(df, classes, sample_size=100):
    # sampling
    df_small = pd.DataFrame()
    for clazz in classes:
        df_clazz = df[df['y'] == clazz]
        df_sample = df_clazz.sample(sample_size)
        df_small = df_small.append(df_sample)

    #shuffle
    df_small = df_small.sample(frac=1.)

    X_small = df_small.drop(columns='y').values
    y_small = df_small['y'].values
    
    return X_small, y_small

In [179]:
#df = pd.DataFrame(X)
#df['y'] = y
#X, y = subsample(df, list(range(10)), sample_size=10)

### Binarize Dataset for Two Chosen Classes

In [180]:
y1 = 1
y2 = 2

ix_y1 = (y == y1)
ix_y2 = (y == y2)

y[ix_y1] = 1
y[ix_y2] = -1

X = X[np.logical_or(ix_y1, ix_y2)]
y = y[np.logical_or(ix_y1, ix_y2)]



In [181]:
print(len(X))
print(y)

2198
[ 1  1  1 ...  1 -1  1]


### Experiment with Perceptron!

In [209]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def basic_run(X_train, X_test, y_train, y_test, kernel, epochs=2):    
    #fit model
    mkp = SpeedyKernelPerceptron(X_train, y_train, kernel)
    mkp.train_for_epochs(epochs, progress=True)
    
    #return errors
    error_train = accuracy_score(y_train, mkp.predict_all(X_train))
    error_test = accuracy_score(y_test, mkp.predict_all(X_test))
    
    return {'err_train': error_train, 'err_test': error_test, 'model': mkp}

In [210]:
np.zeros(3) == 0

array([ True,  True,  True])

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

basic_run(X_train, X_test, y_train, y_test, polynomial_kernel(3), 10)










  0%|          | 0/10 [00:00<?, ?it/s]








 20%|██        | 2/10 [00:00<00:00, 17.76it/s]








100%|██████████| 10/10 [00:00<00:00, 59.68it/s]

{'err_train': 1.0,
 'err_test': 0.9954545454545455,
 'model': <perceptrons.SpeedyKernelPerceptron at 0x1a10ec49b0>}

In [192]:
FR = np.zeros(36)
FR[23] = 1.
FR[13] = 1.
FR[35] = -2.
FR.nonzero()

(array([13, 23, 35]),)

In [171]:
np.zeros(3).nonzero()[0].size == 0

True

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

epochs = 10
kp = SpeedyKernelPerceptron(X_train, y_train, polynomial_kernel(3))
for epoch in list(range(epochs)):
    error_train = accuracy_score(y_train, kp.predict_all(X_train))
    error_test = accuracy_score(y_test, kp.predict_all(X_test))
    kp.run_training_epoch()
    print('Training Accuracy:', error_train)
    print('Test Accuracy:', error_test)

Training Accuracy: 0.5745164960182025
Test Accuracy: 0.5886363636363636
Training Accuracy: 0.9994311717861206
Test Accuracy: 1.0
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727
Training Accuracy: 1.0
Test Accuracy: 0.9977272727272727


In [93]:
kp.past_alphas

[<1x1758 sparse matrix of type '<class 'numpy.float64'>'
 	with 18 stored elements in Compressed Sparse Row format>,
 <1x1758 sparse matrix of type '<class 'numpy.float64'>'
 	with 4 stored elements in Compressed Sparse Row format>]

In [51]:
kp = KernelPerceptron(X_train, y_train, polynomial_kernel(3))


In [96]:
kp.predict_all(X_test)

array([ 1,  1,  1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,  1,  1,  1,
        1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1, -1,
       -1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1, -1,  1,
        1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1, -1,  1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1, -1, -1,
        1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,
        1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1,
        1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,
       -1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1,
       -1,  1,  1,  1,  1, -1,  1, -1, -1,  1,  1, -1,  1, -1,  1, -1, -1,
        1,  1, -1,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,
        1,  1,  1, -1,  1

In [53]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [55]:
len(y)

1269

In [56]:
y.sum()

1269

In [109]:
from scipy.sparse import coo_matrix

In [117]:
x = csr_matrix(np.zeros(9000))
y = csr_matrix(np.random.rand(9000))

In [118]:
%timeit x.multiply(y)

143 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [132]:
%timeit kp.predict_training_pt(0)

538 µs ± 9.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [135]:
538 * 2000

1076000

In [136]:
1076000/1000000

1.076